# Neural Nets t1
In which I attempt some structure.

In [ ]:
%matplotlib widget
#%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# import Importing_Notebooks
import numpy as np
import matplotlib.pyplot as plt
import pickle

A network built of components which:
1. accept an ordered set of reals (we'll use `numpy.array`, and  call them vectors) at the input port and produce another at the output port - this is forward propagation. ${\displaystyle f\colon \mathbf {R} ^{n}\to \mathbf {R} ^{m}}$
1. accept an ordered set of reals at the output port, representing the gradient of the loss function at the output, and produce the gradient of the loss function at the input port - this is back propagation, aka backprop. ${\displaystyle b\colon \mathbf {R} ^{m}\to \mathbf {R} ^{n}}$
1. from the gradient of the loss function at the output, calculate the partial of the loss function w.r.t the internal parameters ${\displaystyle \frac{\partial E}{\partial w} }$
1. accept a scalar $\alpha$ to control the adjustment of internal parameters. _Or is this effected by scaling the loss gradient before passing??_
1. update internal parameters ${\displaystyle w \leftarrow w - \alpha \frac{\partial E}{\partial w} }$


In [ ]:
class Layer:
    def __init__(self):
        pass
    
    def __call__(self, x):
        """Computes response to input"""
        raise NotImplementedError
        
    def backprop(self, output_delE):
        """Uses output error gradient to adjust internal parameters, and returns gradient of error at input"""
        raise NotImplementedError

A network built of a cascade of layers:

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.alpha = 0.1 #FIXME
        
    def extend(self, net):
        self.layers.append(net)
        
    def __call__(self, input):
        v = input
        for net in self.layers:
            v = net(v)
        return v
    
    def learn(self, facts):
        for (x, expected) in facts:
            y = self(x)
            e = y - expected
            loss = e.dot(e)/2.0
            agrad = e * self.alpha
            for net in reversed(self.layers):
                agrad = net.backprop(agrad)
        return loss

## Useful Layers

### Identify

In [ ]:
class IdentityLayer(Layer):
    def __call__(self, x):
        return x
    
    def backprop(self, output_delE):
        return output_delE

### Affine
A layer that does an [affine transformation](https://mathworld.wolfram.com/AffineTransformation.html) aka affinity, which is the classic fully-connected layer with output offsets.

$$ \mathbf{M} \mathbf{x} + \mathbf{b} = \mathbf{y} $$
where
$$
\mathbf{x} = \sum_{j=1}^{n} x_j \mathbf{\hat{x}}_j \\
\mathbf{b} = \sum_{i=1}^{m} b_i \mathbf{\hat{y}}_i \\
\mathbf{y} = \sum_{i=1}^{m} y_i \mathbf{\hat{y}}_i
$$
and $\mathbf{M}$ can be written
$$
\begin{bmatrix}
    m_{1,1} & \dots & m_{1,n} \\
    \vdots & \ddots & \vdots \\
    m_{m,1} & \dots & m_{m,n}
\end{bmatrix} \\
$$

#### Error gradient back-propagation
$$ 
\begin{align}
 \frac{\partial loss}{\partial\mathbf{x}}
  = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{x}}
  = \mathbf{M}\frac{\partial loss}{\partial\mathbf{y}}
\end{align}
$$
_SOLVE: Left-multiply or right-multiply?_

#### Parameter adjustment
$$
 \frac{\partial loss}{\partial\mathbf{M}}
 = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{M}}
 = \frac{\partial loss}{\partial\mathbf{y}} \mathbf{x} \\
 \frac{\partial loss}{\partial\mathbf{b}}
 = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{b}}
 = \frac{\partial loss}{\partial\mathbf{y}}
$$

In [ ]:
class AffinityLayer(Layer):
    """An affine transformation, which is the classic fully-connected layer with offsets"""
    def __init__(self, n, m):
        self.M = np.empty((m, n))
        self.b = np.empty(m)
        self.randomize()
        
    def randomize(self):
        self.M[:] = np.random.randn(*self.M.shape)
        self.b[:] = np.random.randn(*self.b.shape)
        
    def __call__(self, x):
        self.input = x
        self.output = self.M @ x + self.b
        return self.output
    
    def backprop(self, output_delE):
        input_delE = self.M @ output_delE
        self.M -= np.einsum('i,j', output_delE, self.input) # use np.outer?
        self.b -= output_delE
        return input_delE

### Map
Maps a scalar function on the inputs, for e.g. activation layers.

In [ ]:
class MapLayer(Layer):
    """Map a scalar function on the input taken element-wise"""
    def __init__(self, fun, dfundx):
        self.vfun = np.vectorize(fun)
        self.vdfundx = np.vectorize(dfundx)
        
    def __call__(self, x):
        return self.vfun(x)
    
    def backprop(self, output_delE):
        input_delE = self.vdfundx(output_delE)

___

## Tests

### One identity layer
See if the wheels turn:

In [ ]:
net = Network()
net.extend(IdentityLayer())
all(net(np.arange(3)) == np.arange(3))

It does not learn, as expected:

In [ ]:
facts = [(np.arange(2*n, 2*n+2), np.arange(2*n+1, 2*n-1, -1)) for n in range(3)]
net.learn(facts)

In [ ]:
net(np.arange(2,4))

### One map layer

In [ ]:
net = Network()
net.extend(MapLayer(lambda x: x+1, lambda d: 1))
all(net(np.arange(3)) == np.arange(3)+1)

It does not learn, as expected:

In [ ]:
net.learn(facts), all(net(np.arange(5)) == np.arange(5)+1), net(np.arange(2,4))

### One affine layer

In [ ]:
net = Network()
net.extend(AffinityLayer(2,2))

In [ ]:
t = net.layers[0]
t.M, t.b

Can it learn the identity transformation?

In [ ]:
# from nnbench import NNBench
from matplotlib.widgets import Slider, Button, RadioButtons

In [ ]:
class NNBench:
    def __init__(self, net, ideal=lambda x:x):
        self.net = net
        self.ideal = ideal
    
    def checkpoint_net(self):
        self.net_checkpoint = pickle.dumps(self.net)
        
    def rollback_net(self):
        self.net = pickle.loads(self.net_checkpoint)
        
    def training_data_gen(self, n):
        """Generate n instances of labelled training data"""
        for i in range(n):
            v = np.random.randn(2)
            yield (v,self.ideal(v))
        
    def plot_learning(self, n):
        from matplotlib import pyplot as plt
        # self.losses = losses = [self.net.learn(fact for fact in self.training_data_gen(n))]
        self.losses = losses = [self.net.learn([fact]) for fact in self.training_data_gen(n)]
        plt.yscale('log')
        plt.plot(range(len(losses)),losses)
        plt.show(block=0)
        
    def knobs_plot_learning(self, n):
        # from matplotlib import pyplot as plt
        fig, ax = plt.subplots()
        plt.subplots_adjust(left=0.25, bottom=0.25)
        #t = np.arange(0.0, 1.0, 0.001)
        a0 = 5
        f0 = 3
        #delta_f = 5.0
        #s = a0 * np.sin(2 * np.pi * f0 * t)
        
        losses = [self.net.learn([fact]) for fact in self.training_data_gen(n)]
        l, = plt.plot(range(len(losses)), losses, lw=2)
        ax.margins(x=0)
        plt.yscale('log')

        axcolor = 'lightgoldenrodyellow'
        axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
        axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

        sfreq = Slider(axfreq, '⍺', 0, 1, valinit=self.net.alpha)
        samp = Slider(axamp, 'Num', 1, 1000, valinit=100, valstep=1)
        
        big = max(losses)
        ax.set_title(f"maxloss:{big}")
    
        iax = plt.axes([0.025, 0.7, 0.15, 0.15])
        def make_iax_image():
            return np.concatenate([np.concatenate((l.M,np.array([l.b])),axis=0) for l in self.net.layers],axis=1)
        def update_iax(img=[iax.imshow(make_iax_image())]):
            img[0].remove()
            img[0] = iax.imshow(make_iax_image())

        
        def update(val,ax=ax,loc=[l]):
            n = int(samp.val)
            self.rollback_net()
            self.net.alpha = sfreq.val
            losses = [self.net.learn([fact]) for fact in self.training_data_gen(n)]
            big = max(losses)
            ax.set_title(f"max loss:{big}")
            loc[0].remove()
            loc[0], = ax.plot(range(len(losses)), losses, lw=2,color='xkcd:blue')
            ax.set_xlim((0,len(losses)))
            ax.set_ylim((min(losses),big))
            update_iax()
            fig.canvas.draw_idle()

        sfreq.on_changed(update)
        samp.on_changed(update)

        resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
        button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


        def reset(event):
            sfreq.reset()
            samp.reset()
        button.on_clicked(reset)

        rax = plt.axes([0.025, 0.5, 0.15, 0.15], facecolor=axcolor)
        radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


        def colorfunc(label):
            #l.set_color(label)
            #fig.canvas.draw_idle()
            pass
        radio.on_clicked(colorfunc)

        plt.show()
        return 'gc protect:', update, reset, colorfunc,sfreq,samp, radio, button

In [ ]:
bench = NNBench(net)
bench.checkpoint_net()

In [ ]:
bench.plot_learning(100)

In [ ]:
bench.ideal = lambda v: np.array([v[1], v[0]])
bench.knobs_plot_learning(100)